In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/TransferLearning
!ls

/content/gdrive/My Drive/TransferLearning
logs			 X_test.pickle	 Y_test.pickle
VGG16model94accuracy.h5  X_train.pickle  Y_train.pickle


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

In [5]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg19
import numpy as np

Using TensorFlow backend.


In [0]:
tensorboard = TensorBoard(log_dir='./VGG19logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [7]:
vgg19_model = vgg19.VGG19()
vgg19_model.summary()

type(vgg19_model)





574717952/574710816 [==============================] - 16s 0us/step






Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________

keras.engine.training.Model

In [8]:
model = Sequential()
#for layer in vgg19_model.layers:
for layer in vgg19_model.layers[:-3]: #model1.layers.pop() not working, so -1
    model.add(layer) # convert to sequantial model
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [9]:

for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(4,activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [10]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

In [11]:
#Fit the model
            model.fit(X_train, Y_train,
            batch_size=32,
            epochs=20,
            validation_data=(X_test,Y_test),
            callbacks=[tensorboard])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1006 samples, validate on 607 samples


Epoch 1/20
1006/1006 [==============================] - 32s 32ms/step - loss: 9.5553 - acc: 0.3827 - val_loss: 8.4785 - val_acc: 0.4662

Epoch 2/20
1006/1006 [==============================] - 18s 18ms/step - loss: 8.7044 - acc: 0.4533 - val_loss: 8.7698 - val_acc: 0.4530
Epoch 3/20
1006/1006 [==============================] - 18s 18ms/step - loss: 8.6264 - acc: 0.4583 - val_loss: 8.3011 - val_acc: 0.4843
Epoch 4/20
1006/1006 [==============================] - 18s 18ms/step - loss: 8.5590 - acc: 0.4632 - val_loss: 8.7790 - val_acc: 0.4547
Epoch 5/20
1006/1006 [==============================] - 18s 18ms/step - loss: 8.5225 - acc: 0.4702 - val_loss: 8.8377 - val_acc: 0.4498
Epoch 6/20
1006/1006 [==============================] - 18s 18ms/step - loss: 8.4296 - acc: 0.4751 - val_loss: 8.5168 - val_acc: 0.4646
Epoch 7/20
1006/1006 [================

In [12]:
#Score 
scores = model.evaluate(X_test, Y_test, verbose=1) 
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

607/607 [==============================] - 7s 11ms/step
Test loss: 8.579517400637883
Test accuracy: 0.4678747933818245


In [13]:
 #precision    recall  f1-score   support
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

607/607 [==============================] - 12s 20ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       150
           1       0.34      0.99      0.51       150
           2       0.78      0.86      0.82       157
           3       0.00      0.00      0.00       150

    accuracy                           0.47       607
   macro avg       0.28      0.46      0.33       607
weighted avg       0.29      0.47      0.34       607



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
